## Change Detection: Harnessing Image Processing

Exploring the methods of **image processing** for detecting changes using the [Change Detection Dataset](https://gitlab.citius.gal/hiperespectral/ChangeDetectionDataset).

### How to Get Started

1. **Download the Dataset**  
   Begin by downloading the dataset from the link above.

2. **Set Up Your File Paths**  
   Update your configuration to reflect your local setup:
   - `original_file_path`: Path to your **original** image file.
   - `changed_file_path`: Path to your **changed** image file.

With these steps complete, you're all set to dive into change detection and unlock insights with precision-driven image analysis! 🚀

In [1]:
import numpy as np
import cv2 as cv
import scipy.io
from skimage import exposure

In [ ]:
# # Define the path to the .raw file
original_file_path = "./inputs/ChangeDetectionDataset-master/bayArea/mat/Bay_Area_2013.mat"
changed_file_path = "./inputs/ChangeDetectionDataset-master/bayArea/mat/Bay_Area_2015.mat"

original_mat = scipy.io.loadmat(original_file_path)
changed_mat = scipy.io.loadmat(changed_file_path)

print(f"Original Image: [shape = {original_mat["HypeRvieW"].shape} / dtype = {original_mat["HypeRvieW"].dtype}]")
print(f"Changed Image:  [shape = {changed_mat["HypeRvieW"].shape} / dtype = {changed_mat["HypeRvieW"].dtype}]")

Original Image: [shape = (600, 500, 224) / dtype = float64]
Changed Image:  [shape = (600, 500, 224) / dtype = float64]


In [3]:
original_image = original_mat["HypeRvieW"]
changed_image = changed_mat["HypeRvieW"]

In [ ]:
h, w, ch = original_image.shape

In [77]:
for i in range(ch):
    cv.imshow("pic", original_image[:,:,i]/ np.max(original_image[:,:,i]))
    cv.waitKey(50)
cv.destroyAllWindows()

In [69]:
blue_wavelength = (380, 500)
green_wavelength = (500, 650)
red_wavelength = (650, 680)

channel_wavelengths = [blue_wavelength, green_wavelength, red_wavelength]

In [70]:
min_wave_length = 380         # * Wavelength range = (380nm, 2510nm)
max_wave_length = 2510        # * Wavelength range = (380nm, 2510nm)

step_size = (max_wave_length - min_wave_length) / ch

I = np.zeros((h, w, 3), dtype=np.float64) # * colored_image

for i, channel_wavelength in enumerate(channel_wavelengths):
     
     start_idx = int((channel_wavelength[0] - 380) / step_size)
     end_idx = int((channel_wavelength[1] - 380) / step_size)
     
     I[:,:,i] = np.sum(original_image[:,:,start_idx:end_idx], axis=2)
     
     I[:,:,i] = I[:,:,i] / np.max(I[:,:,i])

     cv.imshow("pic", I[:,:,i])
     cv.waitKey(0)

original_colored = (255 * I).astype(np.uint8)

cv.imshow("pic", original_colored)
cv.waitKey(0)
cv.destroyAllWindows()     

In [71]:
h, w, ch = changed_image.shape

min_wave_length = 380         # * Wavelength range = (380nm, 2510nm)
max_wave_length = 2510        # * Wavelength range = (380nm, 2510nm)

step_size = (max_wave_length - min_wave_length) / ch

I = np.zeros((h, w, 3), dtype=np.float64) # * colored_image

for i, channel_wavelength in enumerate(channel_wavelengths):
     
     start_idx = int((channel_wavelength[0] - 380) / step_size)
     end_idx = int((channel_wavelength[1] - 380) / step_size)
     
     I[:,:,i] = np.sum(changed_image[:,:, start_idx:end_idx+1], axis=2)
     
     I[:,:,i] = I[:,:,i]  / np.max(I[:,:,i])
     
     cv.imshow("pic", I[:,:,i])
     cv.waitKey(0)

changed_colored = (255 * I).astype(np.uint8)

cv.imshow("pic", changed_colored)
cv.waitKey(0)
cv.destroyAllWindows()

In [72]:
adjusted_original_colored = exposure.rescale_intensity(original_colored, in_range=(0,100))
adjusted_changed_colored = exposure.rescale_intensity(changed_colored, in_range=(0,100))

    
cv.imshow("pic", adjusted_original_colored)
cv.waitKey(0)
cv.imshow("pic", adjusted_changed_colored)
cv.waitKey(0)
cv.destroyAllWindows() 

cv.imwrite(f"./outputs/P1/bayArea_2013_RGB.png", adjusted_original_colored)
cv.imwrite(f"./outputs/P1/bayArea_2015_RGB.png", adjusted_changed_colored)

True

Calculating the changes in water and vegetation

In [88]:
water_wavelength = [(400, 800)]
vegetation_wavelength = [(750, 900), (1500,1750), (2000, 2320)]

array = [water_wavelength, vegetation_wavelength]
names = ["Water Changes", "Vegetation Changes"]

In [92]:
for idx, item in enumerate(array):

    I1 = np.zeros((h, w), dtype=np.float64)  # * original image
    I2 = np.zeros((h, w), dtype=np.float64)  # * changed image


    for i, wavelength_range in enumerate(item):

        start_idx = int((wavelength_range[0] - 380) / step_size)
        end_idx = int((wavelength_range[1] - 380) / step_size)

        I1 = I1 + np.sum(original_image[:, :, start_idx : end_idx + 1], axis=2)
        I2 = I2 + np.sum(changed_image[:, :, start_idx : end_idx + 1], axis=2)

    # Normalizing the Image
    I1 = (255 * (I1 / np.max(I1))).astype(np.uint8)
    I2 = (255 * (I2 / np.max(I2))).astype(np.uint8)

    cv.imshow("pic", np.concatenate([I1, I2], axis=1))
    cv.waitKey(0)

    final_image = cv.absdiff(I1, I2)

    final_image = exposure.rescale_intensity(final_image, in_range=(0, 100))

    cv.imshow("pic", final_image)
    cv.waitKey(0)
    cv.imwrite(f"./outputs/P1/{names[idx]}.png", final_image)
    cv.destroyAllWindows()